In [76]:
import numpy as np

In [77]:
def add(A, B):
  n = len(A)
  C = [[None for _ in range(n)] for _ in range(n)]
  for i in range(n):
    for j in range(n):
      C[i][j] = A[i][j] + B[i][j]
  return C


In [78]:
def sub(A, B):
  n = len(A)
  C = [[None for _ in range(n)] for _ in range(n)]
  for i in range(n):
    for j in range(n):
      C[i][j] = A[i][j] - B[i][j]
  return C

In [141]:
def strassen(A, B):
  n = len(A)
  if n == 1:
    return [[A[0][0] * B[0][0]]]
  if n == 2:
    return np.dot(A, B).tolist()

  C = [[None for _ in range(n)] for _ in range(n)]

  k = n // 2

  if n % 2 == 0:
    A11 = [row[:k] for row in A[:k]]
    A12 = [row[k:] for row in A[:k]]
    A21 = [row[:k] for row in A[k:]]
    A22 = [row[k:] for row in A[k:]]

    B11 = [row[:k] for row in B[:k]]
    B12 = [row[k:] for row in B[:k]]
    B21 = [row[:k] for row in B[k:]]
    B22 = [row[k:] for row in B[k:]]

    M1 = strassen(add(A11, A22), add(B11, B22))
    M2 = strassen(add(A21, A22), B11)
    M3 = strassen(A11, sub(B12, B22))
    M4 = strassen(A22, sub(B21, B11))
    M5 = strassen(add(A11, A12), B22)
    M6 = strassen(sub(A21, A11), add(B11, B12))
    M7 = strassen(sub(A12, A22), add(B21, B22))

    C11 = add(sub(add(M1, M4), M5), M7)
    C12 = add(M3, M5)
    C21 = add(M2, M4)
    C22 = add(sub(add(M1, M3), M2), M6)

    for i in range(k):
        for j in range(k):
            C[i][j] = C11[i][j]
            C[i][j + k] = C12[i][j]
            C[i + k][j] = C21[i][j]
            C[i + k][j + k] = C22[i][j]

  else:
    A = np.array(A)
    B = np.array(B)

    right_vector = np.dot(A[:-1,:], B[:,-1]).tolist()
    down_vector = np.dot(A[-1], B[:,:-1]).tolist()
    corner_element = np.dot(A[-1], B[:, -1]).item()

    C_ = strassen(A[:-1, :-1].tolist(), B[:-1, :-1].tolist())

    for i in range(n - 1):
        for j in range(n - 1):
            C_[i][j] += A[i,-1].item()*B[-1,j].item()

    for i in range(n - 1):
        for j in range(n - 1):
            C[i][j] = C_[i][j]

    for i in range(n - 1):
        C[i][n - 1] = right_vector[i]

    for j in range(n - 1):
        C[n - 1][j] = down_vector[j]



    C[-1][-1] = corner_element

  return C

In [142]:
A = [
    [1, 2, 3, 4, 5],
    [1, 2, 3, 4, 5],
    [1, 2, 3, 4, 5],
    [1, 2, 3, 4, 5],
    [1, 2, 3, 4, 5]
]

B = [
    [1, 2, 3, 4, 5],
    [1, 2, 3, 4, 5],
    [1, 2, 3, 4, 5],
    [1, 2, 3, 4, 5],
    [1, 2, 3, 4, 5]
]



In [143]:
print(strassen(A, B))

[[15, 30, 45, 60, 75], [15, 30, 45, 60, 75], [15, 30, 45, 60, 75], [15, 30, 45, 60, 75], [15, 30, 45, 60, 75]]


In [144]:
print(np.dot(A, B).tolist())

[[15, 30, 45, 60, 75], [15, 30, 45, 60, 75], [15, 30, 45, 60, 75], [15, 30, 45, 60, 75], [15, 30, 45, 60, 75]]
